# XGBoosting

Leitura dos datasets tratados

In [16]:
import sklearn as skl
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re # Used in Data Processing
from sklearn import preprocessing

allowed_nans = ['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN',
                '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NA',
                'NULL', 'NaN', 'n/a', 'nan', 'null']

treino = pd.read_csv('datasets/treinoTratado.csv', na_values=allowed_nans, keep_default_na=False)
teste = pd.read_csv('datasets/testeTratado.csv', na_values=allowed_nans, keep_default_na=False)

## Construção e modelação

In [17]:

label_encoder = preprocessing.LabelEncoder() 

treino['Date']= label_encoder.fit_transform(treino['Date'])
teste['Date']= label_encoder.fit_transform(teste['Date'])

treino['Month']= label_encoder.fit_transform(treino['Month'])
teste['Month']= label_encoder.fit_transform(teste['Month'])

treino['Season']= label_encoder.fit_transform(treino['Season'])
teste['Season']= label_encoder.fit_transform(teste['Season'])

treino['weather_description']= label_encoder.fit_transform(treino['weather_description'])
teste['weather_description']= label_encoder.fit_transform(teste['weather_description'])

# Para o xgb temos que ter a feature target encoded tbm 
treino['Injeçao na rede (kWh)']= label_encoder.fit_transform(treino['Injeçao na rede (kWh)'])

In [18]:
X_treino = treino.drop('Injeçao na rede (kWh)', axis=1)
y_treino = treino['Injeçao na rede (kWh)']

X_teste = teste

In [19]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

xgb_estimator = xgb.XGBClassifier(random_state=2023)

params =  {'n_estimators': [10, 50, 100], 'learning_rate': [0.1, 0.3, 0.5], 'max_depth': list(range(1,15))}
grid = GridSearchCV(estimator=xgb_estimator,
                    param_grid=params,
                    cv=10,
                    n_jobs=1,
                    verbose=2)
grid.fit(X_treino, y_treino)
grid.best_score_, grid.best_params_

Fitting 10 folds for each of 126 candidates, totalling 1260 fits
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=10; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1, n_estimators=50; total time=   0.0s
[CV] END ....learning_rate=0.1, max_depth=1,

(0.8474071583167258,
 {'learning_rate': 0.3, 'max_depth': 1, 'n_estimators': 50})

In [20]:
predictions = grid.predict(X_teste)

predictions = label_encoder.inverse_transform(predictions)
predict_data = {'Result': predictions}
df = pd.DataFrame(predict_data)
df.index += 1

df.to_csv(f'predictions/xgbPrediction.csv')